## Colab - Install citepro

In [ ]:
!uv pip install --system "git+https://github.com/ptggenomics/citepro"

## Import citepro

In [ ]:
import citepro

## Downloading example dataset

In this tutorial, we are downloading a demo dataset directly to the colab virtual machine. Alternatively, one can upload the count matrix to Google Drive and mount the drive to access the data 

In [ ]:
!curl -LO https://ptgngsdata.s3.us-west-2.amazonaws.com/counts/E44_1_restPBMC_DCpos_filtered_feature_bc_matrix.h5

## Parameters for create_mudata

These are parameters for create_mudata:
  * path_count - file path to the count matrix
  * allow_file - file that contains barcodes passed filter \(a.k.a allow list\)
  * sample_id - a sample id string. \(Optional\)
  * prot_norm - method of protein normalization \(acceptable values: asinh, clr, none\)
  * celltypist_model - predefined model to predict cell types
  * add_3d_umap - whether to generate 3d umap 

#### Explore the celltypist models

Execute the following code to see all the available built-in models from the [Celltypist](https://www.celltypist.org/models)

In [ ]:
import celltypist as ct
ct.models.models_description()


If any of above model fits your need, use it as the `celltypist_model` parameter, otherwize choose none to omit celltype prediction.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:

create_mu_params  = citepro.nbgui.generate_create_mu_setting()

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

### execute the create_mudata

Now let's read in this data and create mudata object. This usually takes >10 minutes on colab, but will be faster \(5-10 minutes\) on a recent generation of HPC

In [ ]:
mudat = citepro.recipe.create_mudata(**create_mu_params)

### Save the result mudata into .h5mu format

In [11]:
!mkdir -p data

In [ ]:

mudat.write_h5mu('data/demo.h5mu')

The MuData can be read back with ```mu.read_h5mu()``` function.


```python
mudat = mu.read_h5mu('data/demo.h5mu')
```

## protein descriptive metadata 
A few useful antibody descriptive data are already incoporated in the data.
  * sum - Sum of all UMIs of this antibody among all cells.
  * percent - percent of total UMI count this antibody occupied. Useful for determining whether a antibody needs furthur titrated. A general rule of thumb is that no antibody should occupy more than 10% of UMI count space, given that this a decently sized \(>20 antibodies\)) cocktail. 
  * median, 75th and 95th - median UMI count across all cells
  

In [ ]:
mudat['prot'].var[['gene_ids', 'sum', 'percent','median', '75th', '95th']].sort_values(by='percent', ascending=False)

## Convert MuData to AnnData
let's convert to anndata for the ease of downstrean analysis

In [14]:
adata = citepro.recipe.mu_to_ann(mudat)

The converted AnnData object has a structure as below:

In [ ]:
adata

## Basic plotting using scanpy

Feature plots using rna-generated umap, colored by predicted cell types

In [ ]:
import scanpy as sc
sc.pl.embedding(adata, basis= 'X_umap_rna', color = ['celltype_ct_majvote'])

Put protein and its encoding transcript expression level side-by-side

In [ ]:
citepro.nbgui.display_multiomics_feature_plot(adata)

Scanpy supports generating 3D umap!

In [ ]:
sc.pl.embedding(adata, basis= 'X_umap_rna_3d', color = ['prot:CD8A.65146.1', 'rna:CD8A'], projection='3d')

## save the processed AnnData object

In [ ]:
adata.write_h5ad('data/demo.h5ad')

Do not forget to download the saved h5ad file using the colab 'File' tab on the left, as all the data on the colab virtual machine will be lost when disconnected. 